In [1]:
# Import Library

import json
import pandas as pd
import re


In [2]:
path_file = "../log_file/LuckyLocker.log"
with open(path_file) as log_report:
    log_splitted = []
                
    for log in log_report:
        splited = log.split('\t')
        splited.pop(5)

        dict = {"timestamp" : splited[0],"process" : splited[1],"pid" : splited[2],"tid" : splited[3],"type" : splited[4], "value" : splited[5],}
        log_splitted.append(dict)


In [3]:
df_log = pd.DataFrame(log_splitted)

In [4]:
log_api = []

for log in log_splitted:
    if(log['type'] == 'ApiCall (U)' and 'Executing' not in log['value'] and 'WerFault.exe' not in log['process']):
        log_api.append(log)
    
df_split_log = pd.DataFrame(log_api)
df_split_log.to_csv("splitted.csv")

In [5]:
#Splitting value column 
apicall_list= []
return_value_list = []

# Check if return value exist
semicolon_delimiter = ';'

for value in df_split_log['value']:
    data_split = value.split(semicolon_delimiter)
        
    #store to dictionary
    apicall_list.append(data_split[0])
    
    if len(data_split) >1:
        return_value_list.append(data_split[1])
    else:
        return_value_list.append('null')


In [6]:
# Log Report Dictionary
log_dct = {
    'apicall' : apicall_list,
    'return_value' : return_value_list
}

In [7]:
df_log_report = pd.DataFrame(log_dct)

In [8]:
apicall_splitted = []

for value in df_log_report['apicall']:
    splitted = value.split('(', 1)
    apicall_splitted.append(splitted)


In [9]:
apicall_splitted[1]

['CreateEvent', 'null)']

In [10]:
list_api = []
list_argument = []

for value in df_log_report['apicall']:
    delete_last_chr = re.sub(".$", '', value)
    split_value = delete_last_chr.split('(', 1)
    list_api.append(split_value[0])
    list_argument.append(split_value[1])

In [11]:
apicall_dct = {
    'api' : list_api,
    'argument' : list_argument
}

In [12]:
df_apicall = pd.DataFrame(apicall_dct)

In [13]:
split_argument = []

for arg in df_apicall['argument']:
    splitted = arg.split(',')
    split_argument.append(splitted)



In [14]:
# # Check longest argument
# for arg in split_argument: 
#     print("Number of Arf : " + str(len(arg)))

In [15]:
# Check longest argument
def longest(list):
    longest_list = max(len(arg) for arg in list)
    return longest_list

In [16]:
print("The longest arg : " + str(longest(split_argument)) + " arguments")

The longest arg : 8 arguments


In [17]:
arg_dict = {}
longest_arg = longest(split_argument)

for i in range(longest_arg):
    index_name = 'argument ' + str(i + 1)
    arg_dict[index_name] = []

arg_dict


{'argument 1': [],
 'argument 2': [],
 'argument 3': [],
 'argument 4': [],
 'argument 5': [],
 'argument 6': [],
 'argument 7': [],
 'argument 8': []}

In [18]:
#Check Every Argument
for i in split_argument:
#     import pdb;pdb.set_trace()
    for j in range(longest_arg):
        index_name = 'argument ' + str(j + 1)
        if len(i) > j: 
            arg_dict[index_name].append(i[j])
        else:
            arg_dict[index_name].append('null')
            

In [19]:
for i in arg_dict.values():
    print(len(i))

70506
70506
70506
70506
70506
70506
70506
70506


In [20]:
df_argument = pd.DataFrame(arg_dict)

In [21]:
df_temp = df_split_log.drop(['value'], axis=1).join(df_apicall.drop(['argument'], axis=1)).join(df_argument).join(df_log_report['return_value'])

pd.set_option("display.max_rows", None, "display.max_columns", None)

log_report_csv = pd.DataFrame(df_temp)
log_report_csv.to_csv('../csv_file/log_report.csv')

In [22]:
print(log_report_csv['api'][0])
log_report_csv.columns


OpenFile


Index(['timestamp', 'process', 'pid', 'tid', 'type', 'api', 'argument 1',
       'argument 2', 'argument 3', 'argument 4', 'argument 5', 'argument 6',
       'argument 7', 'argument 8', 'return_value'],
      dtype='object')

In [23]:
log_report_csv = log_report_csv[:100]

In [24]:
# fill null wit nothing
log_report_csv = log_report_csv.replace('null', 'Nothing', regex=True)

In [25]:
print(log_report_csv['argument 1'][0])


C:\Program Files\Sandboxie-Plus\LogApiDll\logapi32.dll.2.config


In [26]:
sequence_dict = {
    'sequence' : [],
    'api start' : [],
    'api list' : [],
    'which argument' :[],
    'which argument index' : []
}
total_argument = longest_arg

In [27]:

index = 0
seq = 1
for api in log_report_csv['api']:
#     index = 0
    api_list = []
    arg_list = []
    arg_index_list = []
#     api_list.append(api)
#     print(api_list)
#     print(api)
    for args in range(total_argument):
        arg = log_report_csv['argument ' + str(args+1)][index]
#         print('=============== Position in ', arg, '===========')
        if arg == 'Nothing':
#             print('arg is "nothing", continue')
#             index += 1
            continue
        for i in range(len(log_report_csv['argument 1'])):
    #         print(i+1)
            if i < index:
#                 print('index', i+1, 'continue')
                continue
    #         print(arg , log_report_csv['argument 1'][i])
            for j in range(total_argument):
#                 print('argument ' + str(j+1))
#                 print(arg, 'compare with', log_report_csv['argument ' + str(j+1)][i])
                if log_report_csv['argument ' + str(j+1)][i]=='Nothing':
    #                 print('NOTHING nih ngab')
                    continue
                if arg==log_report_csv['argument ' + str(j+1)][i]:
                    api_list.append(log_report_csv['api'][i])
                    arg_list.append('argument ' + str(j+1))
                    arg_index_list.append(i)
#                     print('SAME value')
#     #                 print(api_list)
#                 else:
#                     print('UNEQUAL value')
            if i+1 ==len(log_report_csv['argument 1']):
#                 print('stuck')
#                 print('sequence :',seq)
#                 print('api_start :', api)
#                 print('api :', api_list)
                sequence_dict['sequence'].append(('sequence ' + str(seq)))
                sequence_dict['api start'].append(api)
                sequence_dict['api list'].append(api_list)
                sequence_dict['which argument'].append(arg_list)
                sequence_dict['which argument index'].append(arg_index_list)
                api_list = []
                arg_list = []
                arg_index_list = []
#                 api_list.append(api)
    seq+=1
    index += 1



In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

df = pd.DataFrame(sequence_dict)


<ipython-input-28-0e1f888bee3a>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [29]:
df

,sequence,api start,api list,which argument,which argument index
0,sequence 1,OpenFile,[OpenFile],[argument 1],[0]
1,sequence 3,QuerySystemInformation,"[QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation]","[argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1, argument 1]","[2, 7, 12, 16, 20, 24, 30, 33, 39, 42, 48, 51, 57, 60, 66, 69, 75, 78, 84, 87, 94, 97]"
2,sequence 4,OpenProcess,[OpenProcess],[argument 1],[3]
3,sequence 4,OpenProcess,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess]","[argument 2, argument 2, argument 2, argument 2, argument 2, argument 2]","[3, 4, 5, 6, 88, 98]"
4,sequence 5,OpenProcess,[OpenProcess],[argument 1],[4]
5,sequence 5,OpenProcess,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess]","[argument 2, argument 2, argument 2, argument 2, argument 2]","[4, 5, 6, 88, 98]"
6,sequence 6,OpenProcess,[OpenProcess],[argument 1],[5]
7,sequence 6,OpenProcess,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess]","[argument 2, argument 2, argument 2, argument 2]","[5, 6, 88, 98]"
8,sequence 7,OpenProcess,[OpenProcess],[argument 1],[6]
9,sequence 7,OpenProcess,"[OpenProcess, OpenProcess, OpenProcess]","[argument 2, argument 2, argument 2]","[6, 88, 98]"


In [30]:
df[['sequence','api list']]

,sequence,api list
0,sequence 1,[OpenFile]
1,sequence 3,"[QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation, QuerySystemInformation]"
2,sequence 4,[OpenProcess]
3,sequence 4,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess]"
4,sequence 5,[OpenProcess]
5,sequence 5,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess, OpenProcess]"
6,sequence 6,[OpenProcess]
7,sequence 6,"[OpenProcess, OpenProcess, OpenProcess, OpenProcess]"
8,sequence 7,[OpenProcess]
9,sequence 7,"[OpenProcess, OpenProcess, OpenProcess]"


In [31]:
df[['sequence','api list']].to_csv('../Result/api sequence.csv')